In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project'

In [5]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/data.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

In [9]:
data.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0


In [11]:
data.shape

(1143, 13)

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [10]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories
from src.project import logger
import zipfile

In [11]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                  params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)   

        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema)
        
        return data_validation_config

In [20]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self) -> bool:
        try:
            validation_status = True
            errors = []
          
            data = pd.read_csv(self.config.unzip_data_dir)

            # 🔹 Normalize column names (strip spaces)
            data.columns = data.columns.str.strip()

            # 🔹 Normalize schema as well
            all_schema = {col.strip(): dtype for col, dtype in self.config.all_schema.items()}

            # 🔹 Get actual dataset dtypes
            all_cols = {col.strip(): str(dtype) for col, dtype in data.dtypes.items()}

            for col, dtype in all_cols.items():
                if col not in all_schema:
                    validation_status = False
                    errors.append(f"Column missing in schema: {col}")
                elif dtype != str(all_schema[col]):
                    validation_status = False
                    errors.append(f"Dtype mismatch for '{col}': expected {all_schema[col]}, got {dtype}")

            # 🔹 Also check if schema columns are missing in dataset
            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                    errors.append(f"Column missing in dataset: {col}")

            # 🔹 Save validation status
            with open(self.config.STATUS_FILE, 'w') as f:
                if validation_status:
                    f.write(f"Data Validation found no errors!\n")
                else:
                    f.write('Data Validation found errors!\n')
                    f.write("Errors:\n")
                    for err in errors:
                        f.write(f"{err}\n")
                f.write(f"Validation Status: {validation_status}")
                        
            return validation_status

        except Exception as e:
            raise e


In [21]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-07 13:14:54,876: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-07 13:14:54,886: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-07 13:14:54,921: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-07 13:14:54,942: INFO: common: created directory at: artifacts]
[2025-09-07 13:14:54,962: INFO: common: created directory at: artifacts/data_validation]


In [14]:
with open(Path('artifacts/data_validation/status.txt'), 'r') as f:
                status = f.read().split()[-1]
                print(type(status))
                

<class 'str'>


In [25]:
import pandas as pd

def clean_csv(input_path: str, output_path: str):
    df = pd.read_csv(input_path)

    # 1. Strip leading/trailing spaces
    df.columns = df.columns.str.strip()

    # 2. Remove double spaces inside names
    df.columns = df.columns.str.replace(r"\s+", " ", regex=True)

    # 3. Fix known typos
    rename_map = {
        "Working capitcal Turnover Rate": "Working capital Turnover Rate",
        'bankrupt?': 'bankrupt'
        # Add more corrections if needed
    }

    df.rename(columns=rename_map, inplace=True)

    # 4. (Optional) Lowercase or replace spaces with underscores
    df.columns = df.columns.str.lower().str.replace(" ", "_")

    # Save cleaned CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Cleaned CSV saved at {output_path}")


In [27]:
clean_csv("artifacts/data_ingestion/data.csv", "artifacts/data_ingestion/clean_dataset.csv")


✅ Cleaned CSV saved at artifacts/data_ingestion/clean_dataset.csv
